In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from skimage import io

In [ ]:
### Folder with photos of an object taken from different distances
image_dir = 'data/photos/series_6'

In [ ]:
### Instead you can extract frames from a video
EXTRACT_FROM_VIDEO = False
if(EXTRACT_FROM_VIDEO):
    from Video_utils import extract_video_frames
    video_path = 'data/videos/backpack_lr.mp4'
    extract_dir = 'extract_video'
    save_each = 5
    extract_video_frames(video_path, extract_dir, save_each)
    image_dir = extract_dir

In [ ]:
### To accelerate computations and avoid RAM overflow you may want to compress your images
### But image compression makes the feature detection more difficult and the algortihm may fail
COMPRESS = False
if(COMPRESS):
    from Video_utils import compress_images
    compressed_dir = image_dir+'_compressed'
    compress_images(image_dir, compressed_dir, factor = 0.5)
    image_dir = compressed_dir

In [ ]:
### Photos must be sorted by distance they were taken from
image_paths = [os.path.join(image_dir, im_name) for im_name in os.listdir(image_dir)]
image_paths.sort()

In [ ]:
### You may want to take just a part of the photos - the closest ones 
### as numerical zoom can crash if the object is too far
image_paths = image_paths[:]

In [ ]:
### Choose bounds of the object which will keep fixed size 
### in dolly-zoom effect
base_image = io.imread(image_paths[0])
h0, w0, c = base_image.shape
base_object_bounds = np.array([[0,0],[w0,h0]])

@interact(x1=(0,w0,10), y1=(0,h0,10), x2=(0,w0,10), y2=(0,h0,10))
def adjust_base_object_position(x1=0, y1=0, x2=w0, y2=h0):
    h = y2-y1 if y2>y1 else 0
    w = x2-x1 if x2>x1 else 0
    
    fig,ax=plt.subplots()
    rect = patches.Rectangle((x1, y1), w, h, linewidth=1, edgecolor='r', facecolor='none')
    ax.imshow(base_image)
    ax.add_patch(rect)
    
    base_object_bounds[0] = np.array([x1,y1])
    base_object_bounds[1] = np.array([x2,y2])

In [ ]:
base_object_bounds

In [ ]:
from DollyZoom_utils import make_frames
from Video_utils import make_video, make_gif

In [ ]:
frame_dir = "video_frames"
make_frames(image_paths, base_object_bounds, frame_dir)

In [ ]:
video_name = "video.avi"
make_video(frame_dir, video_name, fps=20)

In [ ]:
# gif_name = "video.gif"
# make_gif(frame_dir, gif_name, fps=20)